In [80]:
grid = '9.1....8.8.5.7..4.2.4....6...7......5..............83.3..6......9................'

In [81]:
rows = 'ABCDEFGHI'
cols = '123456789'

In [82]:
def cross(a, b):
    return [s+t for s in a for t in b]

In [83]:
boxes = cross(rows, cols)

In [84]:
assignments = []

In [85]:
def assign_value(values, box, value):
    """
    Please use this function to update your values dictionary!
    Assigns a value to a given box. If it updates the board record it.
    """
    values[box] = value
    if len(value) == 1:
        assignments.append(values.copy())
    return values

In [86]:
# Define diagonal units of a sudoku
diagonal_units = [[x+y for x, y in zip(rows, cols)], [x+y for x, y in zip(rows, cols[::-1])]]
# And refresh unitlist and peers
row_units = [cross(r,cols) for r in rows]
col_units = [cross(rows,c) for c in cols]
square_units = [cross(rs,cs) for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')]
unitlist = row_units +  col_units + square_units + diagonal_units
units = dict((s, [u for u in unitlist if s in u]) for s in boxes)
peers = dict((s, set(sum(units[s],[]))-set([s])) for s in boxes)

In [87]:
def display(values):
    """
    Display the values as a 2-D grid.
    Input: The sudoku in dictionary form
    Output: None
    """
    width = 1 + max(len(values[s]) for s in boxes)
    line = '+'.join(['-'*(width*3)]*3)
    for r in rows:
        print(''.join(values[r+c].center(width)+('|' if c in '36' else '')
                      for c in cols))
        if r in 'CF': print(line)
    return

In [88]:
def grid_values(grid):
    """Convert grid string into {<box>: <value>} dict with '.' value for empties.

    Args:
        grid: Sudoku grid in string form, 81 characters long
    Returns:
        Sudoku grid in dictionary form:
        - keys: Box labels, e.g. 'A1'
        - values: Value in corresponding box, e.g. '8', or '.' if it is empty.
    """
    lst = list(enumerate(grid))
    ind = list(enumerate(boxes))
    num = '123456789'
    for i in range(len(lst)):
        if lst[i][1] == '.':
            lst[i] = (i,num)
    val = dict((k[1], [v[1] for v in lst][k[0]]) for k in ind)
    return val

In [89]:
def eliminate(values):
    """Eliminate values from peers of each box with a single value.

    Go through all the boxes, and whenever there is a box with a single value,
    eliminate this value from the set of values of all its peers.

    Args:
        values: Sudoku in dictionary form.
    Returns:
        Resulting Sudoku in dictionary form after eliminating values.
    """
    solved_values = [box for box in values.keys() if len(values[box]) == 1]
#    print(solved_values)
    
    for box in solved_values:
        digit = values[box]
        for peer in peers[box]:
            values[peer] = values[peer].replace(digit,'')
    
    return values

In [90]:
def only_choice(values):
    """Finalize all values that are the only choice for a unit.

    Go through all the units, and whenever there is a unit with a value
    that only fits in one box, assign the value to this box.

    Input: Sudoku in dictionary form.
    Output: Resulting Sudoku in dictionary form after filling in only choices.
    """
    for unit in unitlist:
#        print('\ncurrent unit \t- {}'.format(unit))
        for digit in '123456789':
#            print('current digit \t- {}'.format(digit))
            dplaces = [box for box in unit if digit in values[box]]
#            print('dplaces in unit \t- {}'.format(dplaces))
#            print('\n')
            if len(dplaces) == 1:
                values[dplaces[0]] = digit
    return values

In [92]:
def naked_twins(values):
    """Eliminate values using the naked twins strategy.
    Args:
        values(dict): a dictionary of the form {'box_name': '123456789', ...}
    Returns:
        values(dict): the values dictionary with the naked twins eliminated from peers.
    """
    # Find all instances of naked twins
    naked_twin_dict = {}
    pair_dict = {}
    
    for unit in unitlist:
        for box in unit:
            # Get box value consists of the 2 numbers (candidate)
            if len(values[box]) == 2:
                for peer in peers:
                    if box in peers.get(box): 
                        if not values[box] in pair_dict:
                            pair_dict[values[box]] = [box]
                        else:
                            if not box in pair_dict[values[box]]:
                                pair_dict[values[box]].append(box)

    # Examine the dictionary to validate the candidates present as
    # naked twin pairs
    for key in pair_dict:
        if len(pair_dict[key]) == 2:
            if not key in naked_twin_dict:
                naked_twin_dict[key] = [unit] 
            else:
                naked_twin_dict[key].append(unit)

#    if len(naked_twin_dict) != 0:
#        print(naked_twin_dict)
#    else:
#        print('There is no twins in the sudoku.')
                    
    # Eliminate the naked twins as possibilities for their peers
    for key in naked_twin_dict:
        for unit in naked_twin_dict[key]:
            for box in unit:
                if values[box] != key:
                    assign_value(values, box, values[box].replace(key[0], ''))
                    assign_value(values, box, values[box].replace(key[1], ''))
                
#    if len(naked_twin_dict) == 0:
#        print("\nCaution: No changes have been made after naked_twins(values)!!!\n")
        
    return values
#display(naked_twins(values))

In [75]:
def reduce_puzzle(values):
    """ Iterate eliminate(), naked_twins() and only_choice(). If at some point, there is a box with no available values, return False.
    If the sudoku is solved, return the sudoku.
    If after an iteration of both functions, the sudoku remains the same, return the sudoku.
    Args:
        values(dict): A sudoku in dictionary form.
    Returns:
        values(dict): The resulting sudoku in dictionary form.
    """
    solved_values = [box for box in values.keys() if len(values[box]) == 1]
    stalled = False
    while not stalled:
        solved_values_before = len([box for box in values.keys() if len(values[box]) == 1])
        values = eliminate(values)
        values = only_choice(values)
        solved_values_after = len([box for box in values.keys() if len(values[box]) == 1])
        stalled = solved_values_before == solved_values_after
        if len([box for box in values.keys() if len(values[box]) == 0]):
            return False
    return values

In [93]:
def search(values):
    "Using depth-first search and propagation, try all possible values."
    # First, reduce the puzzle using the previous function
    values = reduce_puzzle(values)
    if values is False:
        return False ## Failed earlier
    if all(len(values[s]) == 1 for s in boxes): 
        return values ## Solved!
    # Choose one of the unfilled squares with the fewest possibilities
    n,s = min((len(values[s]), s) for s in boxes if len(values[s]) > 1)
#    print('n - {},  s - {}, values_s - {}'.format(n,s,values[s]))
    # Now use recurrence to solve each one of the resulting sudokus, and 
    for value in values[s]:
        new_sudoku = values.copy()
        new_sudoku[s] = value
        attempt = search(new_sudoku)
        if attempt:
            return attempt

In [95]:
def solve(grid):
    """
    Find the solution to a Sudoku grid.
    Args:
        grid(string): a string representing a sudoku grid.
            Example: '2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3'
    Returns:
        The dictionary representation of the final sudoku grid. False if no solution exists.
    """
    values = grid_values(grid)
    values = reduce_puzzle(values)
    values = search(values)
    values = naked_twins(values)
    return values

if __name__ == '__main__':
    diag_sudoku_grid = '2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3'
    display(solve(diag_sudoku_grid))
    
    try:
        from visualize import visualize_assignments
        visualize_assignments(assignments)
    except:
        print('We could not visualize your board due to a pygame issue. Not a problem! It is not a requirement.')


2 6 7 |9 4 5 |3 8 1 
8 5 3 |7 1 6 |2 4 9 
4 9 1 |8 2 3 |5 7 6 
------+------+------
5 7 6 |4 3 8 |1 9 2 
3 8 4 |1 9 2 |6 5 7 
1 2 9 |6 5 7 |4 3 8 
------+------+------
6 4 2 |3 7 9 |8 1 5 
9 3 5 |2 8 1 |7 6 4 
7 1 8 |5 6 4 |9 2 3 
We could not visualize your board due to a pygame issue. Not a problem! It is not a requirement.


Uploading submission...
[=========================== 100% ===========================] 11507/11507

Waiting for results...Done!

Results:
--------
Not all of the tests passed.  Let me see what I can do to help.

Your submission failed a test that sees whether the naked_twins method
applies the 'naked twins' heuristic.  Your naked_twins function missed
a set of naked_twins.  Try the following example and see what happens:

{"G7": "1234568", "G6": "9", "G5": "35678", "G4": "23678", "G3":
"245678", "G2": "123568", "G1": "1234678", "G9": "12345678", "G8":
"1234567", "C9": "13456", "C8": "13456", "C3": "4678", "C2": "68",
"C1": "4678", "C7": "13456", "C6": "368", "C5": "2", "A4": "5", "A9":
"2346", "A8": "2346", "F1": "123689", "F2": "7", "F3": "25689", "F4":
"23468", "F5": "1345689", "F6": "23568", "F7": "1234568", "F8":
"1234569", "F9": "1234568", "B4": "46", "B5": "46", "B6": "1", "B7":
"7", "E9": "12345678", "B1": "5", "B2": "2", "B3": "3", "C4": "9",
"B8": "8", "B9": "9", "I9": "1235678", "I8": "123567", "I1": "123678",
"I3": "25678", "I2": "123568", "I5": "35678", "I4": "23678", "I7":
"9", "I6": "4", "A1": "2468", "A3": "1", "A2": "9", "A5": "3468",
"E8": "12345679", "A7": "2346", "A6": "7", "E5": "13456789", "E4":
"234678", "E7": "1234568", "E6": "23568", "E1": "123689", "E3":
"25689", "E2": "123568", "H8": "234567", "H9": "2345678", "H2":
"23568", "H3": "2456789", "H1": "2346789", "H6": "23568", "H7":
"234568", "H4": "1", "H5": "35678", "D8": "1235679", "D9": "1235678",
"D6": "23568", "D7": "123568", "D4": "23678", "D5": "1356789", "D2":
"4", "D3": "25689", "D1": "123689"}

Details are available in sudoku-result-31616.json.

Ok. I'm trying.

In [96]:
def solve(grid):
    """
    Find the solution to a Sudoku grid.
    Args:
        grid(string): a string representing a sudoku grid.
            Example: '2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3'
    Returns:
        The dictionary representation of the final sudoku grid. False if no solution exists.
    """
    values = {"G7": "1234568", "G6": "9", "G5": "35678", "G4": "23678", "G3":"245678", "G2":"123568", "G1": "1234678", "G9": "12345678", "G8":"1234567", "C9": "13456", "C8": "13456", "C3": "4678", "C2": "68","C1": "4678", "C7": "13456", "C6": "368", "C5": "2", "A4": "5", "A9":"2346", "A8": "2346", "F1": "123689", "F2": "7", "F3": "25689", "F4":"23468", "F5": "1345689", "F6": "23568", "F7": "1234568", "F8":"1234569", "F9": "1234568", "B4": "46", "B5": "46", "B6": "1", "B7":"7", "E9": "12345678", "B1": "5", "B2": "2", "B3": "3", "C4": "9","B8": "8", "B9": "9", "I9": "1235678", "I8": "123567", "I1": "123678","I3": "25678", "I2": "123568", "I5": "35678", "I4": "23678", "I7":"9", "I6": "4", "A1": "2468", "A3": "1", "A2": "9", "A5": "3468","E8": "12345679", "A7": "2346", "A6": "7", "E5": "13456789", "E4":"234678", "E7": "1234568", "E6": "23568", "E1": "123689", "E3":"25689", "E2": "123568", "H8": "234567", "H9": "2345678", "H2":"23568", "H3": "2456789", "H1": "2346789", "H6": "23568", "H7":"234568", "H4": "1", "H5": "35678", "D8": "1235679", "D9": "1235678","D6": "23568", "D7": "123568", "D4": "23678", "D5": "1356789", "D2":"4", "D3": "25689", "D1": "123689"}
#    values = grid_values(grid)
    values = reduce_puzzle(values)
    values = search(values)
    values = naked_twins(values)
    return values

if __name__ == '__main__':
    diag_sudoku_grid = '2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3'
    display(solve(diag_sudoku_grid))
    
    try:
        from visualize import visualize_assignments
        visualize_assignments(assignments)
    except:
        print('We could not visualize your board due to a pygame issue. Not a problem! It is not a requirement.')


8 9 1 |5 3 7 |2 6 4 
5 2 3 |4 6 1 |7 8 9 
7 6 4 |9 2 8 |1 3 5 
------+------+------
3 4 2 |7 1 5 |8 9 6 
1 5 6 |8 9 3 |4 7 2 
9 7 8 |2 4 6 |5 1 3 
------+------+------
2 1 7 |6 5 9 |3 4 8 
4 3 9 |1 8 2 |6 5 7 
6 8 5 |3 7 4 |9 2 1 
We could not visualize your board due to a pygame issue. Not a problem! It is not a requirement.


And what is the difference? Both of them have been giving the good result!

But when I simplify naked_twins(values) without checking the peers (that i think is not correct) like this

In [101]:
def naked_twins(values):
    """Eliminate values using the naked twins strategy.
    Args:
        values(dict): a dictionary of the form {'box_name': '123456789', ...}
    Returns:
        values(dict): the values dictionary with the naked twins eliminated from peers.
    """
    # Find all instances of naked twins
    naked_twin_dict = {}
    for unit in unitlist:
        # Build a dictionary for identified naked twin pair
        pair_dict = {}
        for box in unit:
            # Identify box - a candidate for a naked twin with 2 numbers
            if len(values[box]) == 2:
                if not values[box] in pair_dict:
                    pair_dict[values[box]] = [box]
                else:
                    pair_dict[values[box]].append(box)
        # Examine the dictionary to validate the candidates present as
        # naked twin pairs
        for key in pair_dict:
            # Condition for the candidate to be a naked twin pair
            if len(pair_dict[key]) == 2:
                if not key in naked_twin_dict:
                    naked_twin_dict[key] = [unit]
                else:
                    naked_twin_dict[key].append(unit)

    # Eliminate the naked twins as possibilities for their peers
    for key in naked_twin_dict:
        for unit in naked_twin_dict[key]:
            for box in unit:
                if values[box] != key:
                    assign_value(values, box, values[box].replace(key[0], ''))
                    assign_value(values, box, values[box].replace(key[1], ''))
    return values


the test was passed and my project was submitted and reviewed successfully

Uploading submission...
[=========================== 100% ===========================] 11008/11008

Waiting for results...Done!

Results:
--------
LGTM!  If your code is well commented, feel free to submit to the
reviewers and see what they have to say.
(https://review.udacity.com/#!/rubrics/689/submit-zip)


Details are available in sudoku-result-31647.json.

If you would like this version of the project to be reviewed,
submit sudoku-31647.zip to the reviews website.